In [28]:
import numpy as np

def initialize_tableau(c, A, b):
    """
    Tạo bảng đơn hình (simplex tableau) từ hàm mục tiêu, ma trận ràng buộc và vector b.
    """
    num_vars = len(c)
    num_constraints = len(b)

    #tableau = np.zeros((num_constraints + 1, num_vars + num_constraints + 1))
    tableau = np.zeros((num_constraints + 1, num_vars + 1))

    # Chèn các hệ số hàm mục tiêu vào hàng cuối cùng
    tableau[-1, :num_vars] = c

    # Thêm các ràng buộc vào tableau
    for i in range(num_constraints):
        tableau[i, :num_vars] = A[i]
        # tableau[i, num_vars + i] = 1  # Biến phụ
        tableau[i, -1] = b[i]

        # Kiểm tra điều kiện vô nghiệm nếu ràng buộc < 0
        if b[i] < 0:
            raise ValueError("Bài toán vô nghiệm: Ràng buộc không khả thi.")

    return tableau

def choose_pivot_column(tableau):
    """
    Chọn cột vào cơ sở (với hệ số âm nhỏ nhất trong hàng hàm mục tiêu).
    """
    if np.all(tableau[-1, :-1] <= 0):
        return None  # Không có cột dương, bài toán đạt tối ưu

    return np.argmax(tableau[-1, :-1])

def choose_pivot_row(tableau, pivot_col):
    """
    Chọn hàng ra khỏi cơ sở dựa trên tỷ lệ.
    """
    ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
    ratios[ratios <= 0] = np.inf  # Loại bỏ các tỷ lệ không hợp lệ
    pivot_row = np.argmin(ratios)

    # Kiểm tra điều kiện không bị chặn
    if np.isinf(ratios[pivot_row]):
        raise ValueError("Bài toán không bị chặn: Không có ràng buộc giới hạn giá trị tối ưu.")

    return pivot_row

def pivot_operation(tableau, pivot_row, pivot_col):
    """
    Thực hiện phép chia hàng trục và biến đổi các hàng khác.
    """
    tableau[pivot_row] /= tableau[pivot_row, pivot_col]

    for i in range(len(tableau)):
        if i != pivot_row:
            tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

def extract_solution(tableau, num_vars):
    """
    Lấy giá trị tối ưu của biến và giá trị tối ưu của hàm mục tiêu.
    """
    solution = np.zeros(num_vars)
    for i in range(len(tableau) - 1):
        basic_var = np.where(tableau[i, :num_vars] == 1)[0]
        if len(basic_var) == 1:
            solution[basic_var[0]] = tableau[i, -1]

    optimal_value = tableau[-1, -1]
    return solution, optimal_value

def print_tableau(tableau, step):
    """
    In ra bảng đơn hình tại mỗi bước.
    """
    print(f"Bảng đơn hình tại bước {step}:")
    print(tableau)
    print("\n")

def simplex(c, A, b):
    """
    Giải bài toán đơn hình sử dụng các hàm trên, kiểm tra các điều kiện khả thi.
    """
    tableau = initialize_tableau(c, A, b)
    step = 0
    print_tableau(tableau, step)

    while True:
        pivot_col = choose_pivot_column(tableau)
        if pivot_col is None:  # Đã đạt tối ưu
            break
        pivot_row = choose_pivot_row(tableau, pivot_col)
        pivot_operation(tableau, pivot_row, pivot_col)
        step += 1
        print_tableau(tableau, step)  # In bảng sau mỗi phép biến đổi

    return extract_solution(tableau, len(c))

# Ví dụ với bài toán tối ưu hóa
'''c = np.array([1, -1, 0, -2, 2, -2])      # Hệ số hàm mục tiêu
A = np.array([[1, 0, 0, 1, 1, -1],
              [0, 1, 0, 1, 0, 1],
              [0, 0, 1, 2, 4, 3]])        # Ma trận hệ số ràng buộc
b = np.array([2, 12, 9])                  # Kết quả các ràng buộc'''

c = np.array([5,4,5,2,1,3])      # Hệ số hàm mục tiêu
A = np.array([[2,4,3,1,0,0],
              [4,2,3,0,1,0],
              [3,0,1,0,0,1]])
              # [0,1,1]])        # Ma trận hệ số ràng buộc
b = np.array([52,60,36])                  # Kết quả các ràng buộc
try:
    solution, optimal_value = simplex(c, A, b)
    print("Giá trị tối ưu của biến:", solution)
    print("Giá trị tối ưu của hàm mục tiêu:", optimal_value)
except ValueError as e:
    print("Lỗi:", e)


Bảng đơn hình tại bước 0:
[[ 2.  4.  3.  1.  0.  0. 52.]
 [ 4.  2.  3.  0.  1.  0. 60.]
 [ 3.  0.  1.  0.  0.  1. 36.]
 [ 5.  4.  5.  2.  1.  3.  0.]]


Bảng đơn hình tại bước 1:
[[  0.           4.           2.33333333   1.           0.
   -0.66666667  28.        ]
 [  0.           2.           1.66666667   0.           1.
   -1.33333333  12.        ]
 [  1.           0.           0.33333333   0.           0.
    0.33333333  12.        ]
 [  0.           4.           3.33333333   2.           1.
    1.33333333 -60.        ]]


Bảng đơn hình tại bước 2:
[[  0.           0.          -1.           1.          -2.
    2.           4.        ]
 [  0.           1.           0.83333333   0.           0.5
   -0.66666667   6.        ]
 [  1.           0.           0.33333333   0.           0.
    0.33333333  12.        ]
 [  0.           0.           0.           2.          -1.
    4.         -84.        ]]


Bảng đơn hình tại bước 3:
[[  0.           0.          -0.5          0.5         -1.

<ipython-input-28-66543bccf82c>:41: RuntimeWarning: divide by zero encountered in divide
  ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]


In [49]:
from scipy.optimize import linprog

# Coefficients of the objective function
c = [3, 2, 1]  # Since we're minimizing 3x1 + 2x2 + x3

# Coefficients for the inequality constraints (Ax >= b form)
# We convert the inequalities to the format required by linprog: Ax <= b by multiplying by -1
A = [
    [-2, 1, -3],
    [-1, -4, 5],
    [-7, 2, -4]
]
b = [-4, 6, -1]

# Bounds for x1, x2, x3 (non-negative constraints)
x_bounds = (0, None)
bounds = [x_bounds, x_bounds, x_bounds]

# Solving the linear programming problem
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')

# Output the result
if result.success:
    print("Optimal solution found:")
    print(f"x1 = {result.x[0]}")
    print(f"x2 = {result.x[1]}")
    print(f"x3 = {result.x[2]}")
    print(f"Minimum value of f(x) = {result.fun}")
else:
    print("No solution found.")


Optimal solution found:
x1 = 0.15384615384615394
x2 = 0.0
x3 = 1.2307692307692306
Minimum value of f(x) = 1.6923076923076925


In [50]:
from scipy.optimize import linprog

# Bài toán ban đầu là cực tiểu hoá: 3x1 + 2x2 + x3
# Trong phương pháp đối ngẫu, ta sẽ chuyển bài toán về cực đại hoá của bài toán đối ngẫu.

# Hệ số hàm mục tiêu của bài toán đối ngẫu (từ b trong bài toán gốc)
c_dual = [-4, 6, -1]

# Ma trận hệ số ràng buộc của bài toán đối ngẫu (A chuyển vị)
A_dual = [
    [2, 1, 7],
    [-1, 4, -2],
    [3, -5, 4]
]

# Hệ số ràng buộc của bài toán đối ngẫu (từ c trong bài toán gốc)
b_dual = [3, 2, 1]

# Ràng buộc không âm của biến trong bài toán đối ngẫu (u1, u2, u3 >= 0)
bounds_dual = [(0, None), (0, None), (0, None)]

# Giải bài toán đối ngẫu
result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual, bounds=bounds_dual, method='highs')

# Xuất kết quả
if result_dual.success:
    print("Giá trị tối ưu của bài toán đối ngẫu tìm được:")
    print(f"u1 = {result_dual.x[0]}")
    print(f"u2 = {result_dual.x[1]}")
    print(f"u3 = {result_dual.x[2]}")
    print(f"Giá trị tối ưu của hàm mục tiêu đối ngẫu = {-result_dual.fun}")
else:
    print("Không tìm thấy nghiệm cho bài toán đối ngẫu.")


Giá trị tối ưu của bài toán đối ngẫu tìm được:
u1 = 1.2307692307692308
u2 = 0.5384615384615384
u3 = 0.0
Giá trị tối ưu của hàm mục tiêu đối ngẫu = 1.6923076923076925
